<a href="https://colab.research.google.com/github/daviscvance/Random/blob/master/colab_attributes_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import subprocess


def get_cpu_attributes():
  """Returns information about the CPU, assuming a linux platform.

  Reads and parses the info in /proc/cpuinfo (on linux platforms) and returns
  information about the processors.

  If summary is set to True, assumes that all processors are identical, returns
  the number of processors, the name of each processor, the number of cores per
  processor, and the speed of each processor.

  Returns:
    A dictionary of key-value pairs stored in /proc/cpuinfo and a special key,
    'summary' continaing summarized information in the following format:
    {
      'summary':{
        'cpu_name': <the cpu name>,
        'cpu_cores': <the number of cores per cpu>,
        'cpu_count': <the total number of cpus>
        'cpu_speed': <the speed per core per cpu>,
      }
      'key':<value>,
      ...
    }
  """
  cpus = subprocess.check_output('cat /proc/cpuinfo', shell=True).split('\n')
  cpus = [re.sub(r'\s', ' ', cpu) for cpu in cpus]
  cpu_attributes = {}
  for param in cpus:
    try:
      key, val = param.split(':')
      key = key.strip()
      val = val.strip()
    except ValueError as e:
      if e.args[0] == 'need more than 1 value to unpack':
        continue
      else:
        raise ValueError(
            'Could not extract key-value pair from cpu parameter {}'.format(
                param
            )
        )
    if key not in cpu_attributes:
      cpu_attributes[key] = []
    cpu_attributes[key].append(val)

  num_cores = cpu_attributes['cpu cores'][0]
  name = cpu_attributes['model name'][0]
  speed = cpu_attributes['cpu MHz'][0]
  num_cpus = len(cpu_attributes['cpu cores'])

  cpu_attributes['summary'] = {
      'cpu_cores': num_cores,
      'cpu_name': name,
      'cpu_speed': speed,
      'cpu_count': num_cpus
  }
  return cpu_attributes


def get_ram_attributes():
  """Returns the amount of ram on the computer where this script is run.

  Assumes the host machine is linux-flavor, and reads ram info from the system
  file /proc/meminfo. This file is parsed, and the total ram, free ram, and unit
  of RAM is returned. If the unit of ram is known to be 'kB' it is converted to
  Gigabytes, else whatever unit is found is returned.

  Returns:
    A dictionary of key-value pairs stored in /proc/meminfo and a special key,
    'summary' continaing summarized information in the following format:
    {
      'summary':{
        'ram_free': <the amount of free ram>,
        'ram_total': <the total amount of ram>,
        'ram_unit': <the unit of storage of ram>
      }
      'key':<value>,
      ...
    }
  """
  ram = subprocess.check_output('cat /proc/meminfo', shell=True).split('\n')

  ram_attributes = {}
  for param in ram:
    try:
      key_val = param.split(':')
      key = key_val[0].strip()
      val = key_val[1].strip()
      ram_attributes[key] = val
    except IndexError:
      pass

  unit = ram_attributes['MemAvailable'].split()[1].strip()
  free = float(ram_attributes['MemAvailable'].split()[0].strip())
  total = float(ram_attributes['MemTotal'].split()[0].strip())

  gigabytes_per_kilobit = 1.25e-7

  if unit == 'kB':
    # assume meaning is kilobit
    free = free * gigabytes_per_kilobit
    total = total * gigabytes_per_kilobit
    unit = 'Gigabyte'

  ram_attributes['summary'] = {
      'ram_free': free,
      'ram_total': total,
      'ram_unit': unit
  }
  return  ram_attributes


In [2]:
get_cpu_attributes()

{'address sizes': ['46 bits physical, 48 bits virtual',
  '46 bits physical, 48 bits virtual'],
 'apicid': ['0', '1'],
 'bogomips': ['4600.00', '4600.00'],
 'bugs': ['cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf',
  'cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf'],
 'cache size': ['46080 KB', '46080 KB'],
 'cache_alignment': ['64', '64'],
 'clflush size': ['64', '64'],
 'core id': ['0', '0'],
 'cpu MHz': ['2300.000', '2300.000'],
 'cpu cores': ['1', '1'],
 'cpu family': ['6', '6'],
 'cpuid level': ['13', '13'],
 'flags': ['fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities',

In [3]:
get_ram_attributes()

{'Active': '532308 kB',
 'Active(anon)': '280692 kB',
 'Active(file)': '251616 kB',
 'AnonHugePages': '0 kB',
 'AnonPages': '298412 kB',
 'Bounce': '0 kB',
 'Buffers': '64572 kB',
 'Cached': '1233740 kB',
 'CommitLimit': '6667636 kB',
 'Committed_AS': '1869516 kB',
 'DirectMap1G': '10485760 kB',
 'DirectMap2M': '5181440 kB',
 'DirectMap4k': '61428 kB',
 'Dirty': '248 kB',
 'HugePages_Free': '0',
 'HugePages_Rsvd': '0',
 'HugePages_Surp': '0',
 'HugePages_Total': '0',
 'Hugepagesize': '2048 kB',
 'Inactive': '1064328 kB',
 'Inactive(anon)': '348 kB',
 'Inactive(file)': '1063980 kB',
 'KernelStack': '3872 kB',
 'Mapped': '150216 kB',
 'MemAvailable': '12638100 kB',
 'MemFree': '11511932 kB',
 'MemTotal': '13335276 kB',
 'Mlocked': '0 kB',
 'NFS_Unstable': '0 kB',
 'PageTables': '4832 kB',
 'SReclaimable': '103480 kB',
 'SUnreclaim': '29156 kB',
 'Shmem': '864 kB',
 'ShmemHugePages': '0 kB',
 'ShmemPmdMapped': '0 kB',
 'Slab': '132636 kB',
 'SwapCached': '0 kB',
 'SwapFree': '0 kB',
 'Swa